In [1]:
# Library Analytics Notebook
"""
Comprehensive library data analysis with Plotly visualizations
Returns JSON data for PyQt5 application integration
"""

import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta, date
import sqlite3
from sqlalchemy import create_engine, func, and_, text
from sqlalchemy.orm import sessionmaker
import json
import warnings
import logging
from typing import Dict, List, Any


project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)
    
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('library_analytics.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class LibraryAnalyticsProcessor:
    """
    Processes library data and generates JSON-serializable analytics
    """
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.engine = create_engine(f'sqlite:///{db_path}')
        logger.info(f"Database connection established: {db_path}")
        
    def load_data(self) -> bool:
        """Load all necessary data from database"""
        try:
            logger.info("Loading data from database...")
            
            # Load patrons data
            patrons_query = """
            SELECT 
                p.user_id, p.patron_id, p.first_name, p.last_name,
                (p.first_name || ' ' || p.last_name) as full_name,
                p.institution, p.grade_level, p.category, p.age, p.gender,
                p.date_of_birth, p.residence, p.phone_number,
                p.membership_status, p.membership_start_date, p.membership_expiry_date,
                p.membership_type
            FROM patrons p
            """
            self.patrons_df = pd.read_sql_query(patrons_query, self.engine)
            
            # Load borrowed books data
            borrowings_query = """
            SELECT 
                bb.borrow_id, bb.user_id, bb.book_id, bb.borrow_date, 
                bb.due_date, bb.return_date, bb.returned, bb.fine_amount,
                b.title as book_title, b.author as book_author, b.class_name as book_genre,
                (p.first_name || ' ' || p.last_name) as patron_name,
                p.category as patron_category, p.membership_status, p.institution,
                p.gender as patron_gender,
                CASE 
                    WHEN bb.return_date IS NULL AND bb.due_date < date('now') 
                    THEN (julianday('now') - julianday(bb.due_date))
                    ELSE 0 
                END as days_overdue
            FROM borrowed_books bb
            JOIN books b ON bb.book_id = b.book_id
            JOIN patrons p ON bb.user_id = p.user_id
            """
            self.borrowings_df = pd.read_sql_query(borrowings_query, self.engine)
            
            # Load payments data
            payments_query = """
            SELECT 
                py.payment_id, py.user_id, py.payment_item_id,
                py.amount_paid, py.total_amount_due, py.payment_date, py.status,
                py.membership_start_date, py.membership_expiry_date, 
                py.is_membership_active, py.notes,
                (p.first_name || ' ' || p.last_name) as patron_name,
                p.category as patron_category, p.gender as patron_gender,
                p.institution as patron_institution,
                pi.name as payment_type, pi.display_name as payment_description,
                pi.is_membership,
                (py.total_amount_due - py.amount_paid) as remaining_amount
            FROM payments py
            JOIN patrons p ON py.user_id = p.user_id
            JOIN payment_items pi ON py.payment_item_id = pi.id
            """
            self.payments_df = pd.read_sql_query(payments_query, self.engine)
            
            # Load attendance data
            attendance_query = """
            SELECT 
                a.id, a.patron_id, a.attendance_date, a.created_at,
                (p.first_name || ' ' || p.last_name) as patron_name,
                p.category as patron_category, p.gender as patron_gender,
                p.institution as patron_institution
            FROM attendances a
            JOIN patrons p ON a.patron_id = p.user_id
            """
            self.attendance_df = pd.read_sql_query(attendance_query, self.engine)
            
            # Convert date columns
            date_columns = ['borrow_date', 'due_date', 'return_date']
            for col in date_columns:
                if col in self.borrowings_df.columns:
                    self.borrowings_df[col] = pd.to_datetime(self.borrowings_df[col])
            
            if 'payment_date' in self.payments_df.columns:
                self.payments_df['payment_date'] = pd.to_datetime(self.payments_df['payment_date'])
            
            if 'attendance_date' in self.attendance_df.columns:
                self.attendance_df['attendance_date'] = pd.to_datetime(self.attendance_df['attendance_date'])
            
            logger.info(f"Data loaded successfully:")
            logger.info(f"  Patrons: {len(self.patrons_df)}")
            logger.info(f"  Borrowings: {len(self.borrowings_df)}")
            logger.info(f"  Payments: {len(self.payments_df)}")
            logger.info(f"  Attendance: {len(self.attendance_df)}")
            
            return True
            
        except Exception as e:
            logger.error(f"Error loading data: {e}")
            return False
    
    def generate_gender_distribution_chart(self) -> Dict[str, Any]:
        """Generate gender distribution chart data"""
        logger.info("Generating gender distribution chart...")
        
        try:
            # Count by gender
            gender_counts = self.patrons_df['gender'].fillna('Unknown').value_counts()
            
            # Create pie chart data
            chart_data = {
                'type': 'pie',
                'title': 'Patron Gender Distribution',
                'labels': gender_counts.index.tolist(),
                'values': gender_counts.values.tolist(),
                'colors': ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'][:len(gender_counts)]
            }
            
            logger.info(f"Gender distribution: {dict(gender_counts)}")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating gender distribution: {e}")
            return {}
    
    def generate_institution_distribution_chart(self) -> Dict[str, Any]:
        """Generate institution distribution chart data"""
        logger.info("Generating institution distribution chart...")
        
        try:
            # Get top 10 institutions
            institution_counts = self.patrons_df['institution'].fillna('Unknown').value_counts().head(10)
            
            chart_data = {
                'type': 'bar',
                'title': 'Top 10 Institutions by Patron Count',
                'x': institution_counts.index.tolist(),
                'y': institution_counts.values.tolist(),
                'color': '#4ECDC4'
            }
            
            logger.info(f"Top institutions: {dict(institution_counts.head(5))}")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating institution distribution: {e}")
            return {}
    
    def generate_borrowing_trends_chart(self) -> Dict[str, Any]:
        """Generate borrowing trends over time"""
        logger.info("Generating borrowing trends chart...")
        
        try:
            # Group by month
            self.borrowings_df['borrow_month'] = self.borrowings_df['borrow_date'].dt.to_period('M')
            monthly_borrowings = self.borrowings_df.groupby('borrow_month').size()
            
            chart_data = {
                'type': 'line',
                'title': 'Borrowing Trends Over Time',
                'x': [str(x) for x in monthly_borrowings.index],
                'y': monthly_borrowings.values.tolist(),
                'color': '#FF6B6B'
            }
            
            logger.info(f"Monthly borrowing trend generated with {len(monthly_borrowings)} data points")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating borrowing trends: {e}")
            return {}
    
    def generate_category_performance_chart(self) -> Dict[str, Any]:
        """Generate patron category performance metrics"""
        logger.info("Generating category performance chart...")
        
        try:
            category_stats = self.borrowings_df.groupby('patron_category').agg({
                'borrow_id': 'count',
                'returned': 'sum',
                'days_overdue': 'mean'
            }).round(2)
            
            chart_data = {
                'type': 'grouped_bar',
                'title': 'Borrowing Performance by Patron Category',
                'categories': category_stats.index.tolist(),
                'series': [
                    {
                        'name': 'Total Borrowings',
                        'data': category_stats['borrow_id'].tolist(),
                        'color': '#4ECDC4'
                    },
                    {
                        'name': 'Books Returned',
                        'data': category_stats['returned'].tolist(),
                        'color': '#96CEB4'
                    }
                ]
            }
            
            logger.info(f"Category performance generated for {len(category_stats)} categories")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating category performance: {e}")
            return {}
    
    def generate_payment_status_chart(self) -> Dict[str, Any]:
        """Generate payment status distribution"""
        logger.info("Generating payment status chart...")
        
        try:
            payment_status_counts = self.payments_df['status'].value_counts()
            
            chart_data = {
                'type': 'doughnut',
                'title': 'Payment Status Distribution',
                'labels': payment_status_counts.index.tolist(),
                'values': payment_status_counts.values.tolist(),
                'colors': ['#96CEB4', '#FFEAA7', '#FF6B6B', '#DDA0DD'][:len(payment_status_counts)]
            }
            
            logger.info(f"Payment status distribution: {dict(payment_status_counts)}")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating payment status: {e}")
            return {}
    
    def generate_daily_attendance_chart(self) -> Dict[str, Any]:
        """Generate daily attendance trends"""
        logger.info("Generating daily attendance chart...")
        
        try:
            # Last 30 days attendance
            end_date = datetime.now().date()
            start_date = end_date - timedelta(days=30)
            
            recent_attendance = self.attendance_df[
                (self.attendance_df['attendance_date'] >= pd.Timestamp(start_date)) &
                (self.attendance_df['attendance_date'] <= pd.Timestamp(end_date))
            ]
            
            daily_attendance = recent_attendance.groupby(
                recent_attendance['attendance_date'].dt.date
            ).size().reindex(
                pd.date_range(start_date, end_date, freq='D').date,
                fill_value=0
            )
            
            chart_data = {
                'type': 'line',
                'title': 'Daily Attendance (Last 30 Days)',
                'x': [str(x) for x in daily_attendance.index],
                'y': daily_attendance.values.tolist(),
                'color': '#45B7D1'
            }
            
            logger.info(f"Daily attendance chart generated for {len(daily_attendance)} days")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating daily attendance: {e}")
            return {}
    
    def generate_revenue_chart(self) -> Dict[str, Any]:
        """Generate revenue trends"""
        logger.info("Generating revenue chart...")
        
        try:
            # Group payments by month
            self.payments_df['payment_month'] = self.payments_df['payment_date'].dt.to_period('M')
            monthly_revenue = self.payments_df.groupby('payment_month')['amount_paid'].sum()
            
            chart_data = {
                'type': 'bar',
                'title': 'Monthly Revenue Trends',
                'x': [str(x) for x in monthly_revenue.index],
                'y': monthly_revenue.values.tolist(),
                'color': '#96CEB4'
            }
            
            logger.info(f"Revenue chart generated for {len(monthly_revenue)} months")
            return chart_data
            
        except Exception as e:
            logger.error(f"Error generating revenue chart: {e}")
            return {}
    
    def generate_all_analytics(self) -> Dict[str, Any]:
        """Generate all analytics and return as JSON-serializable dict"""
        logger.info("Generating comprehensive analytics...")
        
        if not self.load_data():
            return {'error': 'Failed to load data'}
        
        analytics_data = {
            'generated_at': datetime.now().isoformat(),
            'summary_stats': {
                'total_patrons': len(self.patrons_df),
                'total_borrowings': len(self.borrowings_df),
                'total_payments': len(self.payments_df),
                'total_attendance': len(self.attendance_df)
            },
            'charts': {
                'gender_distribution': self.generate_gender_distribution_chart(),
                'institution_distribution': self.generate_institution_distribution_chart(),
                'borrowing_trends': self.generate_borrowing_trends_chart(),
                'category_performance': self.generate_category_performance_chart(),
                'payment_status': self.generate_payment_status_chart(),
                'daily_attendance': self.generate_daily_attendance_chart(),
                'revenue_trends': self.generate_revenue_chart()
            }
        }
        
        logger.info("All analytics generated successfully!")
        return analytics_data
    
    def save_analytics_to_json(self, output_path: str = 'library_analytics.json') -> bool:
        """Save analytics to JSON file"""
        try:
            analytics_data = self.generate_all_analytics()
            
            with open(output_path, 'w') as f:
                json.dump(analytics_data, f, indent=2, default=str)
            
            logger.info(f"Analytics saved to {output_path}")
            return True
            
        except Exception as e:
            logger.error(f"Error saving analytics: {e}")
            return False

# Main execution function
def run_analytics(db_path: str, output_path: str = None) -> Dict[str, Any]:
    """
    Main function to run analytics and return results
    Can be called from external Python scripts
    """
    processor = LibraryAnalyticsProcessor(db_path)
    analytics_data = processor.generate_all_analytics()
    
    if output_path:
        processor.save_analytics_to_json(output_path)
    
    return analytics_data

# For notebook execution
if __name__ == "__main__":
    # Example usage
    DB_PATH = "/home/tjselevani/Desktop/Apps/vscode/python/library_system/library_system.db"  # Update this path
    
    # Run analytics
    processor = LibraryAnalyticsProcessor(DB_PATH)
    analytics = processor.generate_all_analytics()
    
    # Save to file
    processor.save_analytics_to_json('library_analytics.json')
    
    # Display summary
    print("\n" + "="*50)
    print("LIBRARY ANALYTICS SUMMARY")
    print("="*50)
    print(f"Generated at: {analytics['generated_at']}")
    print(f"Total Patrons: {analytics['summary_stats']['total_patrons']}")
    print(f"Total Borrowings: {analytics['summary_stats']['total_borrowings']}")
    print(f"Total Payments: {analytics['summary_stats']['total_payments']}")
    print(f"Charts Generated: {len(analytics['charts'])}")
    print("="*50)

2025-09-06 10:48:56,164 - INFO - Database connection established: /home/tjselevani/Desktop/Apps/vscode/python/library_system/library_system.db
2025-09-06 10:48:56,167 - INFO - Generating comprehensive analytics...
2025-09-06 10:48:56,169 - INFO - Loading data from database...
2025-09-06 10:48:56,277 - INFO - Data loaded successfully:
2025-09-06 10:48:56,280 - INFO -   Patrons: 4
2025-09-06 10:48:56,282 - INFO -   Borrowings: 1
2025-09-06 10:48:56,284 - INFO -   Payments: 4
2025-09-06 10:48:56,286 - INFO -   Attendance: 5
2025-09-06 10:48:56,291 - INFO - Generating gender distribution chart...
2025-09-06 10:48:56,322 - INFO - Gender distribution: {'Female': np.int64(2), 'Male': np.int64(2)}
2025-09-06 10:48:56,324 - INFO - Generating institution distribution chart...
2025-09-06 10:48:56,330 - INFO - Top institutions: {'Greenwood Primary': np.int64(1), 'Lakeside University': np.int64(1), 'Community Center': np.int64(1), 'KCL': np.int64(1)}
2025-09-06 10:48:56,333 - INFO - Generating borr


LIBRARY ANALYTICS SUMMARY
Generated at: 2025-09-06T10:48:56.290804
Total Patrons: 4
Total Borrowings: 1
Total Payments: 4
Charts Generated: 7
